In [96]:
import requests 
import os
import pickle
import time
import pandas as pd

In [65]:
def pickle_out(name, obj, default_path='./sub_result/'):
    pickle_out = open(default_path + name + '.pickle','wb')
    pickle.dump(obj, pickle_out)
    pickle_out.close()
    return

def pickle_in(name, default_path='./sub_result/'):
    pickle_in = open(default_path + name + '.pickle','rb')
    obj = pickle.load(pickle_in)
    pickle_in.close()
    return obj

In [463]:
desc_dir = './dataset/KGRec-dataset/descriptions/'
tags_dir = './dataset/KGRec-dataset/tags/'

# Dictionary for track_id -> description 

In [464]:
desc_dict = {}
for root, dirs, files in os.walk(desc_dir):
    for f in files:
        full_path = root + f
        with open(full_path, 'r') as read_file:
            lines = read_file.readlines()
            lines = ' '.join(lines)
            lines = lines.replace('\n', '')
            text = lines[:4000]
            text = text.replace('#', '')
            text = text.replace(':', '')
            text = text.replace('`', '')
            text = text.replace('@', '')
            desc_dict[int(f.split('.')[0])] = text

# Dictionary for track_id -> tags

In [465]:
tags_dict = {}
for root, dirs, files in os.walk(tags_dir):
    for f in files:
        full_path = root + f
        with open(full_path, 'r') as read_file:
            lines = read_file.readlines()
            if lines:
                lines = ' '.join(lines)
                text = lines[:1500]
                text = text.replace('#', '')
                text = text.replace(':', '')
                text = text.replace('`', '')
                text = text.replace('@', '')
                tags_dict[int(f.split('.')[0])] = text

In [466]:
pickle_out('desc_dict', desc_dict)
pickle_out('tags_dict', tags_dict)

In [467]:
track_id_list = sorted(list(set(desc_dict.keys()) | set(tags_dict.keys())))
pickle_out('track_id_list', track_id_list)
print(f'# of total tracks: {len(track_id_list)}')

# of total tracks: 8640


In [468]:
merged_text_dict = {}
for idx in track_id_list:
    merged_text = ''
    if idx in desc_dict:
        merged_text += desc_dict[idx]
    if idx in tags_dict:
        merged_text += tags_dict[idx]
    merged_text_dict[idx] = merged_text

In [469]:
pickle_out('merged_text_dict', merged_text_dict)

In [475]:
# babelnet user api key
key1 = 'aad4522a-5d4a-44a9-8547-294ec523dab9'
key2 = '6ff8c462-16a7-478c-b2e6-e70dc48fe678'
key3 = '6f14fa68-12e9-487c-af7d-759dc64ef2eb'
key4 = '7c1c1ae5-deee-4b02-882c-14f1ec04feef'
key5 = '93ba3e99-19db-42e3-9c39-132ad80c0720'
key6 = '9bb94618-1028-48b8-ac3d-9eac001ed668'
key7 = '9cdfc33e-3661-48df-a02c-4283b1c09504'
key8 = 'a2565707-778e-41d3-801d-bed548c81b2e'
key9 = 'f1fac4d0-ad75-4b07-87e5-d0eb2d997c60'

t1 = track_id_list[:1000]
t2 = track_id_list[1000:2000]
t3 = track_id_list[2000:3000]
t4 = track_id_list[3000:4000]
t5 = track_id_list[4000:5000]
t6 = track_id_list[5000:6000]
t7 = track_id_list[6000:7000]
t8 = track_id_list[7000:8000]
t9 = track_id_list[8000:]

request_list = \
[(key1, t1),  (key2, t2),  (key3, t3),
 (key4, t4),  (key5, t5),  (key6, t6),
 (key7, t7),  (key8, t8),  (key9, t9)]

In [472]:
babelnet_dict = {}

[1, 2, 3, 4, 5]

In [486]:
URL_prefix = 'https://babelfy.io/v1/disambiguate?text='
for (key, t) in request_list:
    for idx in t:
        if not idx in babelnet_dict:
            try:
                text = merged_text_dict[idx]
                URL = URL_prefix + text + '&lang=en&key=' + key
                r = requests.get(url = URL, headers={'Accept-Encoding': 'identity'})
                babelnet_dict[idx] = r
                if idx % 100 == 0:
                    print(f'{idx} idx is being processed.')
                time.sleep(0.05)
            except BaseException as e:
                print(e, idx)

800 idx is being processed.
900 idx is being processed.
1000 idx is being processed.
HTTPSConnectionPool(host='babelfy.io', port=443): Max retries exceeded with url: /v1/disambiguate?text=This%20features%20the%20R&B%20singer-songwriter%20Ne-Yo%20.%20Jadakiss%20explained%20to%20MTV%20News%20that%20this%20song%20is%20%20for%20the%20ladies%20.%20''%20He%20added%20.%20%20We%20trying%20to%20make%20big%20music%20.%20We%20're%20trying%20to%20do%20it%20big%20for%20the%20ladies%20.%20-LSB-%20The%20women%20-RSB-%20still%20gon%20na%20cop%20before%20they%20get%20their%20hair%20and%20nails%20done%20,%20so%20we%20got%20ta%20cater%20to%20them%20.%20We%20gon%20na%20always%20show%20love%20to%20the%20streets%20,%20but%20we%20got%20ta%20show%20the%20ladies%20we%20acknowledging%20them%20this%20year%20.%20Heavy%20!%20''kiss%20USA%20deutsch%20jadakiss%20New-York-City%20rap%20NYC%20LOX%20chillin%20hip-hop%20ny%20rnb%20new-york%20rabbit%20america%20schrockstar%20JipJop%2050-Cent%20Jada%20by-my-side%20american

7800 idx is being processed.
7900 idx is being processed.
8000 idx is being processed.
8100 idx is being processed.
8200 idx is being processed.
8300 idx is being processed.
8400 idx is being processed.
8500 idx is being processed.
8600 idx is being processed.


In [518]:
non_ok_key = []
for key in track_id_list:
    if (not key in babelnet_dict)\
        or (not babelnet_dict[key].ok)\
        or (not isinstance(babelnet_dict[key].json(), list)):
        non_ok_key.append(key)

In [517]:
URL_prefix = 'https://babelfy.io/v1/disambiguate?text='

for i, idx in enumerate(non_ok_key):
    try:
        text = merged_text_dict[idx]
        URL = URL_prefix + text + '&lang=EN&key=' + key
        r = requests.get(url = URL, headers={'Accept-Encoding': 'identity'})
        babelnet_dict[idx] = r
        if i % 100 == 0:
            print(f'{i} idx is being processed.')
        time.sleep(0.1)
    except BaseException as e:
        print(e, idx)

0 idx is being processed.


In [521]:
pickle_out('babelnet_dict_redo', babelnet_dict)

In [455]:
pickle_out('babelnet_dict', babelnet_dict)